# Unit 4: Neighborhood-based Collaborative Filtering for Rating Prediction

In this section we generate personalized recommendations for the first time. We exploit rating similarities among users and items to identify similar users and items that assist in finding the relevant items to recommend for each user.

This describes the fundamental idea behind Collaborative Filtering (CF) and using kNN is a neighborhood-based approach towards CF. In a later unit we will also have a look at model-based approaches.

This is also the first time we try to predict user ratings for unknown items using rating predictions to take the top-$N$ items with the highest rating predictions and recommend those to the user.

In [1]:
from collections import OrderedDict
import itertools
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd

In [2]:
from recsys_training.data import Dataset
from recsys_training.evaluation import get_relevant_items

In [3]:
ml100k_ratings_filepath = '../../data/raw/ml-100k/u.data'

## Load Data

In [4]:
data = Dataset(ml100k_ratings_filepath)
data.rating_split(seed=42)
user_ratings = data.get_user_ratings()

The idea behind this recommender is to use item ratings of the $k$ most similar users (neighbors). We identify those _nearest neighbors_ with a similarity metric which we apply to the ratings both, root user and possible neighbor, have in common. Similarity thereby means having a similar opinion on movies.

The steps are as follows:

1. Compute user-user similarities (we use the Pearson Correlation Coefficient here, but feel free to try other similarity metrics)

2. For each user:

    1. Get the k nearest neighbors along with their similarities
    2. Collect the neighborhood item ratings and ignore those already rated by the root user
    3. Item Rating Prediction: Compute the similarity-weighted sum of neighborhood item ratings
    4. Recommendations: Get the $N$ items with the highest ratings that have a minimum rating count

### 1. User-User Similarities

In [5]:
sim_metric = 'pearson'
user_user_sims = {}

In [6]:
def get_entity_sim(a: int, b: int,
                   entity_ratings: Dict[int, float],
                   metric: str = 'pearson') -> tuple:
    """
    Cosine Similarity
    Pearson Correlation
    Adjusted Cosine Similarity
    Jaccard Similarity (intersection over union) - not a good idea as it does not incorporate ratings, e.g.
        even the same users have rated two items, highest Jaccard similarity as evidence for high item similarity,
        their judgement may be very differently on the two items, justifying dissimilarity
    """
    # 1. isolate e.g. users that have rated both items (a and b)
    key_intersection = set(entity_ratings[a].keys()).intersection(entity_ratings[b].keys())
    ratings = np.array([(entity_ratings[a][key], entity_ratings[b][key]) for key in key_intersection])
    n_joint_ratings = len(ratings)

    if n_joint_ratings > 1:
        # 2. apply a similarity computation technique
        if metric == 'pearson':
            # Warning and nan if for one entity the variance is 0
            try:
                sim = np.corrcoef(ratings, rowvar=False)[0, 1]
            # if the standard deviation of variables becomes 0
            except RuntimeWarning:
                sim = None
        elif metric == 'cosine':
            nom = ratings[:, 0].dot(ratings[:, 1])
            denom = np.linalg.norm(ratings[:, 0]) * np.linalg.norm(ratings[:, 1])
            sim = nom / denom
        elif metric == 'euclidean':
            sim = normalized_euclidean_sim(ratings[:, 0], ratings[:, 1])
        elif metric == 'adj_cosine':
            sim = None
        else:
            raise ValueError(f"Value {metric} for argument 'mode' not supported.")
    else:
        sim = None

    return sim, n_joint_ratings

In [7]:
user_pairs = itertools.combinations(data.users, 2)

This takes a few seconds to finish ...

In [8]:
for pair in user_pairs:
    user_user_sims[pair] = get_entity_sim(pair[0], pair[1],
                                          user_ratings,
                                          sim_metric)

/anaconda3/envs/recsys_training/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/anaconda3/envs/recsys_training/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [9]:
user_user_sims[(1,4)]

(0.9759000729485333, 5)

## 2. Computing Recommendations

### A. Implement Nearest Neighbors for a given user

**Task:** It's your turn again. Complete `get_k_nearest_neighbors` to return a sorted list of the $k$ nearest neighbors - identified by their id - for a given user, each along with its similarity.

In [10]:
def get_k_nearest_neighbors(user: int, k: int, user_user_sims: dict) -> List[Tuple[int, float]]:
    neighbors = set(data.users)
    neighbors.remove(user)

    nearest_neighbors = dict()
    for neighbor in neighbors:
        sim = user_user_sims[tuple(sorted((user, neighbor)))][0]
        if pd.notnull(sim):
            nearest_neighbors[neighbor] = sim

    nearest_neighbors = sorted(nearest_neighbors.items(),
                               key=lambda kv: kv[1],
                               reverse=True)
    
    return nearest_neighbors[:k]

In [11]:
user_neighbors = get_k_nearest_neighbors(1, k=10, user_user_sims=user_user_sims)

In [12]:
user_neighbors

[(107, 1.0),
 (443, 1.0),
 (485, 1.0),
 (687, 1.0),
 (791, 1.0),
 (820, 1.0),
 (34, 0.9999999999999999),
 (240, 0.9999999999999999),
 (281, 0.9999999999999999),
 (384, 0.9999999999999999)]

### B. Obtain the Neighborhood Ratings

**Task:** Now, use the nearest neighbors and get their ratings, but leave out the items our root user has already rated. Return a mapping from unknown item to a list of dicts with neighbor similarity and item rating.

In [13]:
def get_neighborhood_ratings(user, user_neighbors: List[Tuple[int, float]]) -> Dict[int, List[Dict[str, float]]]:
    neighborhood_ratings = {}
    for neighbor, sim in user_neighbors:
        neighbor_ratings = user_ratings[neighbor].copy()
        
        # collect neighbor ratings and items
        for item, rating in neighbor_ratings.items():
            add_item = {'sim': sim, 'rating': rating}
            if item not in neighborhood_ratings.keys():
                neighborhood_ratings[item] = [add_item]
            else:
                neighborhood_ratings[item].append(add_item)
        
    # remove known items
    known_items = list(user_ratings[user].keys())
    for known_item in known_items:
        neighborhood_ratings.pop(known_item, None)
    
    return neighborhood_ratings

In [14]:
neighborhood_ratings = get_neighborhood_ratings(1, user_neighbors)

In [15]:
list(neighborhood_ratings.items())[:10]

[(340,
  [{'sim': 1.0, 'rating': 5.0},
   {'sim': 1.0, 'rating': 5.0},
   {'sim': 0.9999999999999999, 'rating': 4.0}]),
 (325, [{'sim': 1.0, 'rating': 3.0}]),
 (288,
  [{'sim': 1.0, 'rating': 3.0},
   {'sim': 1.0, 'rating': 3.0},
   {'sim': 1.0, 'rating': 4.0},
   {'sim': 1.0, 'rating': 3.0},
   {'sim': 1.0, 'rating': 5.0},
   {'sim': 0.9999999999999999, 'rating': 5.0}]),
 (312,
  [{'sim': 1.0, 'rating': 4.0}, {'sim': 0.9999999999999999, 'rating': 4.0}]),
 (313,
  [{'sim': 1.0, 'rating': 2.0},
   {'sim': 1.0, 'rating': 4.0},
   {'sim': 1.0, 'rating': 5.0},
   {'sim': 1.0, 'rating': 5.0},
   {'sim': 0.9999999999999999, 'rating': 5.0},
   {'sim': 0.9999999999999999, 'rating': 5.0}]),
 (300,
  [{'sim': 1.0, 'rating': 1.0},
   {'sim': 0.9999999999999999, 'rating': 3.0},
   {'sim': 0.9999999999999999, 'rating': 4.0},
   {'sim': 0.9999999999999999, 'rating': 4.0}]),
 (264,
  [{'sim': 1.0, 'rating': 3.0},
   {'sim': 1.0, 'rating': 3.0},
   {'sim': 1.0, 'rating': 3.0}]),
 (333,
  [{'sim': 1.0,

### C. Compute Rating Predictions from Neighborhood Ratings

**Task:** In this step, we estimate ratings for the seed user based on the neighborhood ratings. We implement a similarity weighted average of neighbor ratings for that. Return a mapping from item to its prediction and the count of neighbor ratings received.

In [16]:
def compute_rating_pred(neighborhood_ratings: dict) -> dict:
    rating_preds = dict()
    for item, ratings in neighborhood_ratings.items():
        if len(ratings) > 0:
            sims = np.array([rating['sim'] for rating in ratings])
            ratings = np.array([rating['rating'] for rating in ratings])
            pred_rating = (sims * ratings).sum() / sims.sum()
            count = len(sims)
            rating_preds[item] = {'pred': pred_rating,
                                  'count': count}
        else:
            rating_preds[item] = {'pred': None, 'count': 0}

    return rating_preds

In [17]:
rating_preds = compute_rating_pred(neighborhood_ratings)

In [18]:
list(rating_preds.items())[:20]

[(340, {'pred': 4.666666666666667, 'count': 3}),
 (325, {'pred': 3.0, 'count': 1}),
 (288, {'pred': 3.8333333333333335, 'count': 6}),
 (312, {'pred': 4.0, 'count': 2}),
 (313, {'pred': 4.333333333333333, 'count': 6}),
 (300, {'pred': 2.9999999999999996, 'count': 4}),
 (264, {'pred': 3.0, 'count': 3}),
 (333, {'pred': 4.0, 'count': 5}),
 (1243, {'pred': 3.0, 'count': 1}),
 (322, {'pred': 2.5, 'count': 2}),
 (305, {'pred': 4.0, 'count': 1}),
 (327, {'pred': 4.0, 'count': 3}),
 (302, {'pred': 4.6, 'count': 5}),
 (687, {'pred': 3.0, 'count': 1}),
 (358, {'pred': 1.0, 'count': 2}),
 (323, {'pred': 2.5, 'count': 2}),
 (286, {'pred': 3.875, 'count': 8}),
 (678, {'pred': 2.0, 'count': 1}),
 (343, {'pred': 4.0, 'count': 2}),
 (644, {'pred': 3.0, 'count': 1})]

### D. Compute the Top-$N$ Recommendation Items

**Task:** The last step takes the rating predictions and returns the $N$ highest predictions which have a minimum rating count.

In [19]:
def compute_top_n(rating_preds: dict, min_count: int, N: int) -> OrderedDict:
    rating_preds = {key: val for (key, val) in rating_preds.items()
                    if val['count'] >= min_count}
    # assuming more ratings mean higher confidence in the prediction
    sorted_rating_preds = sorted(rating_preds.items(),
                                 key=lambda kv: (kv[1]['pred'], kv[1]['count']),
                                 reverse=True)

    return OrderedDict(sorted_rating_preds[:N])

In [20]:
top_n_recs = compute_top_n(rating_preds, min_count=2, N=10)

In [21]:
top_n_recs

OrderedDict([(242, {'pred': 5.0, 'count': 2}),
             (340, {'pred': 4.666666666666667, 'count': 3}),
             (332, {'pred': 4.666666666666667, 'count': 3}),
             (302, {'pred': 4.6, 'count': 5}),
             (690, {'pred': 4.5, 'count': 2}),
             (313, {'pred': 4.333333333333333, 'count': 6}),
             (333, {'pred': 4.0, 'count': 5}),
             (327, {'pred': 4.0, 'count': 3}),
             (312, {'pred': 4.0, 'count': 2}),
             (343, {'pred': 4.0, 'count': 2})])

### Combine all steps in a single method `get_recommendations`

In [22]:
def get_recommendations(user: int,
                        user_user_sims: dict,
                        k: int,
                        min_count: int,
                        N: int):
    user_neighbors = get_k_nearest_neighbors(user, k=k, user_user_sims=user_user_sims)
    neighborhood_ratings = get_neighborhood_ratings(user, user_neighbors)
    rating_preds = compute_rating_pred(neighborhood_ratings)
    top_n_recs = compute_top_n(rating_preds, min_count=min_count, N=N)
    return top_n_recs

In [23]:
get_recommendations(1, user_user_sims, 10, 2, 10)

OrderedDict([(242, {'pred': 5.0, 'count': 2}),
             (340, {'pred': 4.666666666666667, 'count': 3}),
             (332, {'pred': 4.666666666666667, 'count': 3}),
             (302, {'pred': 4.6, 'count': 5}),
             (690, {'pred': 4.5, 'count': 2}),
             (313, {'pred': 4.333333333333333, 'count': 6}),
             (333, {'pred': 4.0, 'count': 5}),
             (327, {'pred': 4.0, 'count': 3}),
             (312, {'pred': 4.0, 'count': 2}),
             (343, {'pred': 4.0, 'count': 2})])

## Evaluation

In [24]:
k = 60
min_count = 10
N = 10

In [25]:
relevant_items = get_relevant_items(data.test_ratings)

In [26]:
users = relevant_items.keys()
prec_at_N = dict.fromkeys(data.users)

for user in users:
    recommendations = get_recommendations(user, user_user_sims, k, min_count, N)
    recommendations = list(recommendations.keys())
    hits = np.intersect1d(recommendations,
                          relevant_items[user])
    prec_at_N[user] = len(hits)/N

In [27]:
np.mean([val for val in prec_at_N.values() if val is not None])

0.08106382978723406